## Fais

Facebook AI Similarity Search(Faiss) is a library for efficient similarity search and clustering of dense vectors

In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

In [4]:
loader = TextLoader("speech.txt")
documents = loader.load()
text_splitter =CharacterTextSplitter(chunk_size=1000,chunk_overlap=30)
docs = text_splitter.split_documents(documents)

In [5]:
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct our

In [9]:
embeddings = OllamaEmbeddings(model="mxbai-embed-large")
db = FAISS.from_documents(docs,embeddings)
db

In [13]:
# ## querying
# query = "What does the speaker believe is the main reason the United States should enter the war?"
query = "How does the speaker describe the desired outcome of the war?"
docs = db.similarity_search(query)
docs

docs[0].page_content


'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

## As a Retriever


In [17]:
retriever = db.as_retriever()
docs = retriever.invoke(query)
docs[0].page_content

'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

## Similarity Search With Score 

In [18]:
docs_and_score = db.similarity_search_with_score(query)
docs_and_score

[(Document(id='10823f74-b3b7-4da1-bdc9-779eca667a3c', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
  np.float32(204.15138)),
 (Document(id='177db138-bb01-482b-b37c-a8feeedca78d', metadata={'source': 'spe

In [19]:
embedding_vector = embeddings.embed_query(query)
embedding_vector


[0.5107148885726929,
 -0.39260798692703247,
 -0.281170129776001,
 0.4477137625217438,
 -0.7403903603553772,
 -1.5128364562988281,
 -0.701414167881012,
 0.3383903205394745,
 0.04697436839342117,
 -0.13547763228416443,
 0.28588607907295227,
 0.052604567259550095,
 -0.33681392669677734,
 0.5879623889923096,
 -0.3940303325653076,
 -0.41355571150779724,
 -0.721683919429779,
 -0.20961643755435944,
 0.2560981810092926,
 0.049583811312913895,
 -0.03378840535879135,
 -0.4084742069244385,
 -0.6067572832107544,
 0.38273587822914124,
 -0.11283763498067856,
 0.27384042739868164,
 -0.5984748601913452,
 0.2037556916475296,
 1.1324883699417114,
 -0.11249327659606934,
 -0.17260892689228058,
 0.5426985621452332,
 -0.040777817368507385,
 -0.6852837204933167,
 0.4342670142650604,
 -0.6312419772148132,
 0.4277639389038086,
 -1.9120969772338867,
 0.13513079285621643,
 -0.15925979614257812,
 0.25277525186538696,
 0.2891911566257477,
 0.5580454468727112,
 -0.4557691812515259,
 -1.4731577634811401,
 -0.2824184

In [21]:
docs_score = db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(id='10823f74-b3b7-4da1-bdc9-779eca667a3c', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='177db138-bb01-482b-b37c-a8feeedca78d', metadata={'source': 'speech.txt'}, page_content='…\n

## Saving and loading

In [22]:
db.save_local("faiss_index")

In [24]:
new_db = FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)

In [25]:
docs = new_db.similarity_search(query)
docs

[Document(id='10823f74-b3b7-4da1-bdc9-779eca667a3c', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='177db138-bb01-482b-b37c-a8feeedca78d', metadata={'source': 'speech.txt'}, page_content='…\n